<h1><b>Will You Survive On Titanic AI</b></h1>
<h2>Importing Libraries and Database</h2>

In [122]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import time

In [158]:
titanic = pd.read_csv(r"titanic.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# sns.pairplot(titanic, hue="Survived")

<h1>Data Cleanup</h1>

In [159]:
titanic = titanic.drop(columns=['Name', 'Cabin','Embarked','Ticket','PassengerId'])
titanic = titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]
titanic.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,3,male,22.0,1,0,7.2500,0
1,1,female,38.0,1,0,71.2833,1
2,3,female,26.0,0,0,7.9250,1
3,1,female,35.0,1,0,53.1000,1
4,3,male,35.0,0,0,8.0500,0


In [101]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row,column]-min1)/(max1-min1)
                x.at[row,column]=xprim 
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [160]:
DataProcessing.shuffle(titanic)

In [103]:


for row in range(len(titanic)-1,-1,-1):
    if titanic.at[row,'Sex']=="male":
        titanic.at[row,'Sex']=1
    elif titanic.at[row,'Sex']=="female":
        titanic.at[row,'Sex']=0
    else:
        titanic = titanic.drop(row,axis=0)

        
# values=titanic.select_dtypes(exclude="object")
# columnNames=values.columns.tolist()
# for column in columnNames:
# column = "PassengerId"
# for row in range(len(titanic)-1,-1,-1):
#     print(titanic.at[row,column])
#     temp = titanic.at[row,column]
#     if isinstance(temp, (int, float, complex)):
#         print(titanic.at[row,column])
        
        
#     else:
#         print(titanic.iloc[row])
#         titanic = titanic.drop(row,axis=0)


titanic.head()


,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,1,1,45.0,1,0,83.4750,0
1,3,1,19.0,0,0,6.7500,0
2,3,0,NaN,0,0,7.7500,1
3,3,1,21.0,0,0,7.8542,0
4,3,0,15.0,0,0,8.0292,1


In [161]:
titanic = titanic.dropna()
titanic = titanic.reset_index(drop=True)
DataProcessing.normalization(titanic)
titanic.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,0.5,female,0.448829,0.0,0.000000,0.025374,1
1,1.0,male,0.160717,0.0,0.333333,0.039525,0
2,1.0,male,0.229613,0.0,0.000000,0.014110,0
3,1.0,male,0.235876,0.0,0.000000,0.015412,0
4,0.0,male,0.336089,0.0,0.000000,0.149765,1


In [142]:
trainingSet, validatingSet = DataProcessing.split(titanic,0.7)

In [143]:
titanic

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,0.0,1,0.448829,0.0,0.000000,0.051505,1
1,0.5,0,0.298509,0.0,0.000000,0.025374,0
2,0.0,1,0.498935,0.0,0.000000,0.060508,1
3,0.0,0,0.361142,0.0,0.000000,0.432884,1
4,1.0,0,0.210823,0.5,0.333333,0.015469,1
...,...,...,...,...,...,...,...
1040,0.5,0,0.711888,0.0,0.000000,0.020495,0
1041,1.0,0,0.348616,0.0,0.000000,0.015412,0
1042,1.0,1,0.198296,0.0,0.000000,0.015713,0
1043,1.0,1,0.436302,0.0,0.000000,0.013907,0


<h2>k-nearest neighbors algorithm</h2>

In [132]:
class Knn:
    @staticmethod
    def knn(x,element,k=1):
        dist = []
        for i in range(len(x)):
            sum = 0
            for j in range(len(element)-1):
                sum += (x.iloc[i,j] - element.iloc[j])**2
            dist.append(sum**(1/2))
        x.loc[:,"dist"]=dist
        y = x.sort_values("dist")
        kElementów = {}
        for i in range(k):
            kElementów[y["Survived"].iat[i]] = 0
        for i in range(k):
            kElementów[y["Survived"].iat[i]] +=1
        return max(kElementów, key=kElementów.get)
    @staticmethod
    def accuracy(x,z,k):
        good = 0
        for i in range(len(z)):
            if( Knn.knn(x,z.iloc[i],k) == z.iloc[i][6] ):
                good +=1
        return good/z.shape[0]

In [136]:
# trainingSet.head()
# start = time.time()
# print(Knn.accuracy(trainingSet,validatingSet,3))
# end = time.time()
# print(end - start)

In [151]:
#knn test
titanic = titanic.dropna()
titanic = titanic.reset_index(drop=True)
DataProcessing.normalization(titanic)
titanic.head()
for i in range(5):
    DataProcessing.shuffle(titanic)
    trainingSet, validatingSet = DataProcessing.split(titanic,0.7)

    start = time.time()
    print(Knn.accuracy(trainingSet, validatingSet,4))
    end = time.time()
    print(end - start)

C:\Users\user\AppData\Local\Temp\ipykernel_3348\1232234049.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:,"dist"]=dist


0.0
58.36804175376892


C:\Users\user\AppData\Local\Temp\ipykernel_3348\1232234049.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:,"dist"]=dist


KeyboardInterrupt: 

In [ ]:
validatingSet.head()

<h2>Naive Bayes Algorithm</h2>

In [110]:
class NaiveBayes:
    @staticmethod
    def mean(column):
        return sum(column)/len(column)
    @staticmethod
    def stv(column, mean1):
        tmp=0
        for i in column:
            tmp+=(i-mean1)**2
        return np.sqrt(tmp/len(column))
    @staticmethod
    def gaussProbability(x, mu, sigma):
        if sigma == 0:
            return 0
        exponent=np.exp(-0.5*(x-mu)**2/(sigma**2))
        return 1/(sigma*np.sqrt(2*np.pi))*exponent
    @staticmethod
    def classify(x, sample):
        probability = []
        for className in x['Survived'].unique().tolist():
            p1 = 1
            columns = x.columns.tolist()[:-1]
            data = x[x['Survived'] == className]
            for column in columns:
                dataFromOneColumn = data[column]
                mu = NaiveBayes.mean(dataFromOneColumn)
                sigma = NaiveBayes.stv(dataFromOneColumn, mu)
                prob = NaiveBayes.gaussProbability(sample[column], mu, sigma)
                p1 *= prob
            p2 = x['Survived'].value_counts()[className]
            probability.append(p1*p2/len(x))
        maxIdx = probability.index(max(probability))
        return x['Survived'].unique().tolist()[maxIdx]
    @staticmethod
    def accuracy(trainSet, valSet):
        positiveCases = 0
        for idx, i in valSet.iterrows():
            if i['Survived'] == NaiveBayes.classify(trainSet, i):
                positiveCases += 1
        return positiveCases / len(valSet)

In [173]:
# naive test
arr = []
for i in range(5):
    DataProcessing.shuffle(titanic)
    arr.append(NaiveBayes.accuracy(trainingSet, validatingSet))
print(arr)
print(sum(arr)/len(arr))

[0.597444089456869, 0.6357827476038339, 0.6357827476038339, 0.6070287539936102, 0.6198083067092651]
0.6191693290734824


knn + pca

In [ ]:
# class PCA:
#     @staticmethod
#     def mean(column):
#         return sum(column)/len(column)
#     @staticmethod
#     def pca(data):
#         columns = x.columns.tolist()[:-1]
#         for column in columns:
#             dataFromOneColumn = data[column]
#             mean = PCA.mean(dataFromOneColumn)
            

In [21]:
# titanic.values[:,:-1].tolist()

[[1.0, 1, 0.36114242765877486, 0.0, 0.0, 0.018510168852370702],
 [0.0, 0, 0.46135538018288863, 0.125, 0.0, 0.17566830077223786],
 [1.0, 1, 0.3987222848553175, 0.0, 0.0, 0.11027245763075773],
 [1.0, 1, 0.19829637980709008, 0.125, 0.3333333333333333, 0.06709553154495196],
 [1.0, 1, 0.008267568583239383, 0.0, 0.1111111111111111, 0.01824998458022693],
 [1.0, 0, 0.36114242765877486, 0.125, 0.1111111111111111, 0.02042143996477265],
 [1.0, 1, 0.8121007140172868, 0.0, 0.0, 0.015126992566498259],
 [0.0, 0, 0.3861956657898033, 0.0, 0.2222222222222222, 0.32179836714362564],
 [1.0, 1, 0.22334961793811847, 0.0, 0.0, 0.015126992566498259],
 [1.0, 1, 0.311035951396718, 0.0, 0.0, 0.015468569817999833],
 [0.0, 1, 0.2859827132656896, 0.0, 0.1111111111111111, 0.12366716556464086],
 [0.0, 1, 0.6116748089690592, 0.0, 0.0, 0.0],
 [0.0, 1, 0.6868345233621446, 0.0, 0.0, 0.05953203526170282],
 [0.0, 0, 0.2358762370036327, 0.0, 0.2222222222222222, 0.051301584996521765],
 [0.5, 1, 0.6743079042966303, 0.0, 0.0, 0

In [42]:
# features = titanic.columns[1:]

# # Compute covariance matrix
# cov_matrix = np.cov(titanic[features].T, bias=True, ddof=0, aweights=np.array([titanic[features]]))

# # Compute eigenvalues and eigenvectors
# eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# # Sort eigenvalues and eigenvectors in descending order
# eigenvectors = eigenvectors[:, np.argsort(-eigenvalues)]
# eigenvalues = eigenvalues[np.argsort(-eigenvalues)]

# # Compute principal components
# principal_components = np.dot(titanic[features], eigenvectors)

# # Create new DataFrame with principal components
# principal_df = pd.DataFrame(data=principal_components, columns=["PC1", "PC2"])

# # Concatenate the new DataFrame with the original one
# result_df = pd.concat([titanic.iloc[:, 0], principal_df], axis=1)

In [138]:
def PCA(X , num_components):
    X_meaned = X - np.mean(X , axis = 0)
    cov_mat = np.cov(X_meaned.astype(float) , rowvar = False)
     
    eigen_values , eigen_vectors = np.linalg.eigh(cov_mat)
     
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:,sorted_index]
     
    eigenvector_subset = sorted_eigenvectors[:,0:num_components]
     
    X_reduced = np.dot(eigenvector_subset.transpose() , X_meaned.transpose() ).transpose()
     
    return X_reduced

In [174]:
x = titanic.iloc[:,0:6]
 
target = titanic.iloc[:,6]
 
mat_reduced = PCA(x , 2)
 
principal_df = pd.DataFrame(mat_reduced , columns = ['PC1','PC2'])
 
principal_df = pd.concat([principal_df , pd.DataFrame(target)] , axis = 1)

## KNN with PCA

In [175]:
titanic = pd.read_csv(r"titanic.csv")
titanic = titanic.drop(columns=['Name', 'Cabin','Embarked','Ticket','PassengerId'])
titanic = titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]
for row in range(len(titanic)-1,-1,-1):
    if titanic.at[row,'Sex']=="male":
        titanic.at[row,'Sex']=1
    elif titanic.at[row,'Sex']=="female":
        titanic.at[row,'Sex']=0
    else:
        titanic = titanic.drop(row,axis=0)
DataProcessing.shuffle(titanic)
titanic = titanic.dropna()
titanic = titanic.reset_index(drop=True)
DataProcessing.normalization(titanic)
titanic.head()

x = titanic.iloc[:,0:6]
 
target = titanic.iloc[:,6]
 
mat_reduced = PCA(x , 2)
 
principal_df = pd.DataFrame(mat_reduced , columns = ['PC1','PC2'])
 
principal_df = pd.concat([principal_df , pd.DataFrame(target)] , axis = 1)


DataProcessing.shuffle(titanic)
trainingSet, validatingSet = DataProcessing.split(titanic,0.7)
start = time.time()
print(Knn.accuracy(trainingSet, validatingSet,4))
end = time.time()
print(end - start)

DataProcessing.shuffle(principal_df)
trainingSetPCA, validatingSetPCA = DataProcessing.split(principal_df,0.7)
start = time.time()
print(Knn.accuracy(trainingSetPCA, validatingSetPCA,4))
end = time.time()
print(end - start)
    
# for i in range(5):
#     DataProcessing.shuffle(titanic)
#     trainingSet, validatingSet = DataProcessing.split(titanic,0.7)

#     start = time.time()
#     print(Knn.accuracy(trainingSet, validatingSet,4))
#     end = time.time()
#     print(end - start)

    
# for i in range(5):
#     DataProcessing.shuffle(principal_df)
#     trainingSetPCA, validatingSetPCA = DataProcessing.split(principal_df,0.7)
    
#     start = time.time()
#     print(Knn.accuracy(trainingSetPCA, validatingSetPCA,4))
#     end = time.time()
#     print(end - start)

C:\Users\user\AppData\Local\Temp\ipykernel_3348\3806468683.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:,"dist"]=dist


0.8274760383386581
94.56679177284241


C:\Users\user\AppData\Local\Temp\ipykernel_3348\3806468683.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:,"dist"]=dist


0.853035143769968
40.792466163635254


## NaiveBayes with PCA


In [177]:
titanic = pd.read_csv(r"titanic.csv")
titanic = titanic.drop(columns=['Name', 'Cabin','Embarked','Ticket','PassengerId'])
titanic = titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]
for row in range(len(titanic)-1,-1,-1):
    if titanic.at[row,'Sex']=="male":
        titanic.at[row,'Sex']=1
    elif titanic.at[row,'Sex']=="female":
        titanic.at[row,'Sex']=0
    else:
        titanic = titanic.drop(row,axis=0)
DataProcessing.shuffle(titanic)
titanic = titanic.dropna()
titanic = titanic.reset_index(drop=True)
DataProcessing.normalization(titanic)
titanic.head()

x = titanic.iloc[:,0:6]
 
target = titanic.iloc[:,6]
 
mat_reduced = PCA(x , 2)
 
principal_df = pd.DataFrame(mat_reduced , columns = ['PC1','PC2'])
 
principal_df = pd.concat([principal_df , pd.DataFrame(target)] , axis = 1)

arr = []
for i in range(5):
    DataProcessing.shuffle(titanic)
    trainingSet, validatingSet = DataProcessing.split(titanic,0.7)
    start = time.time()
    arr.append(NaiveBayes.accuracy(trainingSet, validatingSet))
    end = time.time()
    print(end - start)
    
print(arr)
print(sum(arr)/len(arr))

arr = []
for i in range(5):
    DataProcessing.shuffle(titanic)
    trainingSetPCA, validatingSetPCA = DataProcessing.split(principal_df,0.7)
    start = time.time()
    arr.append(NaiveBayes.accuracy(trainingSet, validatingSet))
    end = time.time()
    print(end - start)
    
print(arr)
print(sum(arr)/len(arr))

1.710251808166504
1.752232551574707
2.399555206298828
1.704007625579834
1.6762945652008057
[0.8402555910543131, 0.8626198083067093, 0.8370607028753994, 0.8115015974440895, 0.8306709265175719]
0.8364217252396167
1.5519890785217285
1.5599992275238037
2.5772852897644043
1.9681310653686523
1.5692811012268066
[0.8306709265175719, 0.8338658146964856, 0.8370607028753994, 0.8242811501597445, 0.8626198083067093]
0.837699680511182
